In [1]:
import numpy as np
import random
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_mat import *
from ids import *
from block_maxvol import *
from block_rect_maxvol import *
import scipy.linalg
from numba import jit
import os
%matplotlib inline

In [ ]:
#### PLUQ TESTING
row_exp_max = 10
col_exp_max = 7
nder = 2
ndim = nder+1
n_iter = int(1e1)
# treshold = 1e-05 # We do not use threshold
dir_str = '.'
saves = dict()
dir_matrix = os.path.join(dir_str, "matrices2")
try:
    os.mkdir(dir_matrix)
except:
    pass

si_count = 0

for i in range(n_iter):
    col_exp = np.random.randint(1,col_exp_max)
    row_exp = np.random.randint(col_exp,row_exp_max)
    A = np.random.rand(row_exp*ndim,col_exp*ndim)
    try:
        p,q,lu,indx = pluq_ids(A,nder,do_pullback=False,pullbacks=4,overwrite_a=False,preserve_order=False)
        u = np.triu(lu[:A.shape[1]])
        s = la.svd(u,compute_uv=False)

        cur_thre = min(s[-1], np.abs(u[-1,-1]))
        try:
            to_save = saves[(row_exp, col_exp)] > cur_thre
        except:
            to_save = True

        if to_save:
        #if (s[-1] < treshold) or (np.abs(u[-1,-1]) < treshold):
            np.savez(os.path.join(dir_matrix,"pluq_row_exp={}_col_exp={}".format(row_exp, col_exp)), A = A)
            saves[(row_exp, col_exp)] = cur_thre

    except SingularError:
        np.savez(os.path.join(dir_matrix,"singular_row_exp={}_col_exp={}_i={}".format(row_exp, col_exp, si_count)), 
                 A = A)
        si_count += 1

### Processing of obtained matrices

In [ ]:
for i in range(1,10):
    for j in range(1,8):
        ### Try to open file with any combination of indices
        try:
            data = np.load('./matrices3/pluq_row_exp={}_col_exp={}.npz'.format(i,j))
            A = data['A']
            Fl = True
        except:
            ### File does not exist, just skip part that is below
            continue
        try:
            p,q,lu,indx = pluq_ids(A,nder,do_pullback=True,pullbacks=20,overwrite_a=False,preserve_order=False)
            perm = A[p][:,q]
            s_A = la.svd(perm[:A.shape[1]],compute_uv=False)
            u = np.triu(lu[:A.shape[1]])
            s = la.svd(u,compute_uv=False)
            print (s_A[-1], s[-1], u[-1,-1], la.det(u), i,j)
        except SingularError as err:
            print 'not full column rank', err.value

In [ ]:
### SINGLE EXAMPLE
data = np.load('./matrices3/pluq_row_exp=6_col_exp=2.npz')
A = data['A']
p,q,lu,indx = pluq_ids(A,nder,do_pullback=False,pullbacks=4,overwrite_a=False,preserve_order=False)
perm = A[p][:,q]
s_A = la.svd(perm[:A.shape[1]],compute_uv=False)
u = np.triu(lu[:A.shape[1]])
s = la.svd(u,compute_uv=False)
print ('Singular values of U = {}'.format(s))
print ('Singular values of quadr PAQ = {}'.format(s_A))
print ('det of U = {}, det of quadr PAQ = {}'.format(la.det(u), la.det(perm[:A.shape[1]])))
print s_A[0]/s[0]

In [ ]:
def AllCombsrect(A, l, num_take_blocks=None, debug=False):
    # INPUT 
    # A -- matrix
    # l -- number of dimensions (derivatives), size of block is l+1
    # num_take_bocks -- blocks returnd by algorithm
    l1 = l + 1
    b_one = np.arange(l1)
    num_of_blocks = A.shape[0] // l1
    max_det = 0.0
    last_sv = []
    volumes = []
    good_idx = None
    if num_take_blocks is None:
        num_take_blocks = A.shape[1] // l1 # Take square mat and test square block maxvol
    for i in itertools.combinations(xrange(num_of_blocks), num_take_blocks):
        # print i
        idx = np.hstack((b_one + j*l1 for j in i))
        Al = A[idx]
        s = la.svd(Al[:Al.shape[1]],compute_uv=False)
        last_sv.append(s[-1])
        det = np.abs(la.det(Al))
        volumes.append(det)
        if det > max_det:
            max_det = det
            good_idx = idx

            
    return max_det, good_idx, last_sv, volumes

In [ ]:
@jit
def statistics_pluq(nder,row_exp,col_exp,n_iter, dir_stats, T_matrix=False):
    ndim = nder+1
    si_count = 0
    ind = []
    ind_vol = []
    for i in range(n_iter):
        if T_matrix:
            x = 2*np.random.rand(row_exp, nder) - 1
            c = GenMat(col_exp*ndim, x,poly=cheb, debug=False,pow_p=1)
            A = matrix_prep(c,nder+1)
        else:    
            A = np.random.rand(row_exp*ndim,col_exp*ndim)
            
        p,q,lu,indx = pluq_ids(A,nder,do_pullback=False,pullbacks=4,overwrite_a=False,preserve_order=False)
        perm = A[p][:,q]
        s_A = la.svd(perm[:A.shape[1]],compute_uv=False)
        v_A = np.abs(la.det(perm[:A.shape[1]]))
            
        top_det,idx,l_sv,l_vol = AllCombsrect(A,nder)

        l_sv = np.sort(np.array(l_sv,dtype=float))
        l_vol = np.sort(np.array(l_vol,dtype=float))

        ### Check the bins, memorize coincedents
        ind_i, = np.where(np.abs(l_sv - s_A[-1]) <= 1e-10)    
        ind.append(ind_i)
        ind_vol_i, = np.where(np.abs(l_vol - v_A) <= 1e-10)    
        ind_vol.append(ind_vol_i) 
        if (i % 1000)== 0 :
            ind_hist,ind_vol_hist = np.histogram(ind),np.histogram(ind_vol)
            np.savez(os.path.join(dir_stats,"stat_row_exp={}_col_exp={},T={}".format(row_exp, col_exp, compute_poly_matrix,i)), ind_hist = ind_hist, ind_vol_hist=ind_vol_hist,i=i)
    return(ind,ind_vol)

In [ ]:
#### STAT PART
row_exp_max = 5
col_exp_max = 3
nder = 2
compute_poly_matrix = True
n_iter = int(2e3)
dir_str = '.'
dir_stats = os.path.join(dir_str, "stat_experiments")
try:
    os.mkdir(dir_stats)
except:
    pass

for col_exp in range(2, col_exp_max+1):
    for row_exp in range(col_exp+1, row_exp_max+1):
        try:
            ind,ind_vol = statistics_pluq(nder,row_exp,col_exp,n_iter,dir_stats, T_matrix=compute_poly_matrix)
        except SingularError as err:
                    continue


In [ ]:
data = np.load('./stat_experiments/stat_row_exp=4_col_exp=3,T=True.npz')

In [ ]:
ind = data['ind_hist']
ind_vol = data['ind_vol_hist']
N_iter = data['i']

In [ ]:
hist, bins = ind_vol
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()

# RGV 2
### Test for linear dependency. 1st thought about pullbacks

In [ ]:
mat_test = np.array([
    [ 1000, 0, 0, 0,0,0 ],
    [0,3,0,0,0,1],
    [0, 0 , 1000, 0, 0, 0] , 
    [0, 0, 0, 2, 0, 0],
    [0, 0, 0, 0, 1000, 0] , 
    [0,3,0,2,0,1],
    [1,2,3,4,0,0],
    [4,3,2,1,0,1]
], dtype=float)

In [ ]:
np.linalg.matrix_rank(mat_test)

## Check, that matrix is good

In [ ]:
np.linalg.det(mat_test[[0,1,4,5,6,7]])

In [ ]:
p,q,lu,info = pluq_ids(mat_test,1, do_pullback=True, debug=True)
print info

# Another Error 
### That was about columns swap. 
If piv[0] == j, piv[1] != j+1

In [ ]:
mat_test_5 = np.array([[18,  7, 10, 14, 19, 12],
       [16, 15,  5,  6, 17, 15],
       [15,  0,  7,  7, 12, 11],
       [12,  4,  6, 13,  6,  5],
       [19,  8, 10, 16,  3, 16],
       [10,  3,  5,  1,  1,  4],
       [ 0,  7,  7,  6, 17,  9],
       [14,  7,  9, 16, 17,  5],
       [18, 18, 19,  3,  9, 12],
       [12,  7, 16,  1,  0,  2]], dtype=float)

In [ ]:
p,q,lu,info = pluq_ids(mat_test_5,1 ,debug=True)

In [ ]:
print(la.det(mat_test_5[p[:6]]))
print p

## Yet another bug
About feature of maxvol's performance. It returns error if we have a long pair consisted from more then 2 elements each with rank < 2

In [ ]:
mat_test_6 = np.array([[ 3, 13,  4, 11,  6, 11],
       [14, 19, 16,  1,  4,  7],
       [17, 14, 18, 18,  9,  6],
       [ 7,  8,  8,  2, 16, 16],
       [10, 19,  3, 10, 18,  5],
       [ 1, 17,  9, 19, 15, 13],
       [13,  1,  4,  1, 17,  9],
       [10, 13,  8, 11,  3,  7],
       [ 4,  8,  6,  0,  1,  8],
       [ 8, 10, 10,  9,  1, 19]], dtype=float)

In [ ]:
p,q,lu, info = pluq_ids(mat_test_6,1,do_pullback=True, debug=True)

## Yet another bug ver. 2.0: Division by Zero
### This and below: again about columns swap. Solved

In [ ]:
mat_test_7 = np.array([[ -5,  14, -17, -18,   4,  19],
       [ -1,   0,  16,  -9,   3, -14],
       [ 12,   0, -16,  -4,  -6,  18],
       [ -9,  -4, -17,  12,  14,   0],
       [  0,   2,   0,   0, -15,   0],
       [ 13,  -5,  -8,  -7,  13, -16],
       [ -5,   0,   3,   7,  -7,   0],
       [ 19,  -1, -10,  18,  17,  -9],
       [-15,   6,  -5, -12,   2,   4],
       [-15, -19, -13, -14,   0,  -1]], dtype=float)

In [ ]:
p,l,u,q, info = pluq_ids(mat_test_7, debug=True)

## Yet another bug ver. 3.0: Division by Zero

In [ ]:
mat_test_8 = np.array([[ -5,  -9, -13,  -2,  19,  11],
       [ 14, -17,  -8,   2, -12,   0],
       [ 17, -19,  -3,   6, -19,   1],
       [ 19,   0, -16, -19,  19,  18],
       [ -5,   2,   1,   4, -17,   1],
       [  4,   2,   7,   9, -10, -15],
       [ 19,  -9,   0, -12,   0,  -6],
       [-10,  15,  12,   2,   2,  13],
       [ 16,   0,   3, -16,  16, -15],
       [-15, -14,   9,   8,   0,  17]], dtype=float)

In [ ]:
wer = np.copy(A[p1][:,q1])
yx = np.zeros((2), dtype=int)
for ind in range(15):
        elimination(wer,ind)
        if (ind > 0) and ((ind+1) % 3 == 0):
            print('block det')
            print la.det(wer[ind-2:ind+1][:,ind-2:ind+1])
            loc_max = np.argmax(np.abs(wer[ind:, ind:]))
            yx[0] = loc_max / (wer.shape[1] - ind)
            yx[1] = loc_max % (wer.shape[1] - ind)
            print wer[ind:,ind:][yx[0],yx[1]]
        qe = wer[12:15][:,12:15]
        print('eliminating index', ind)
        print(wer[ind,ind])
        print('last square rank', la.matrix_rank(qe))
        _,s,_ = scipy.linalg.svd(qe)
        print ('last square last singular',s[2])
        print ('whole square matrix rank',la.matrix_rank(wer[:15]))
        _,s1,_ = scipy.linalg.svd(wer[:15])
        print ('whole square last singular',s1[14])
        print ('-------------')